### Lab 03

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
sc.stop()

spark = SparkSession.builder \
    .appName("Map-Reduce-Examples") \
    .master("spark://873bad4e62fe:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/14 06:28:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
post = ["Learning #BigData with #PySpark is fun! #AI",
"#AI is transforming the world. #BigData #MachineLearning",
"I love #PySpark and #BigData. #AI #DataScience",
"#DataScience and #AI are the future. #BigData",
"#PySpark is awesome! #BigData #AI"]
post_rdd = sc.parallelize(post)

In [14]:
#Extract Hashtags: Use flatMap to extract all hashtags from the posts.
def get_hashtags(post):
    return [word for word in post.split() if word.startswith("#")]
hashtags_rdd = post_rdd.flatMap(get_hashtags)
hashtags_rdd.collect()


['#BigData',
 '#PySpark',
 '#AI',
 '#AI',
 '#BigData',
 '#MachineLearning',
 '#PySpark',
 '#BigData.',
 '#AI',
 '#DataScience',
 '#DataScience',
 '#AI',
 '#BigData',
 '#PySpark',
 '#BigData',
 '#AI']

In [15]:
# Map Hashtags to Pairs: Use map to transform each hashtag into a key-value pair (hashtag, 1).
tuples_hashtags_rdd = hashtags_rdd.map(lambda x: (x, 1))
tuples_hashtags_rdd.collect()

[('#BigData', 1),
 ('#PySpark', 1),
 ('#AI', 1),
 ('#AI', 1),
 ('#BigData', 1),
 ('#MachineLearning', 1),
 ('#PySpark', 1),
 ('#BigData.', 1),
 ('#AI', 1),
 ('#DataScience', 1),
 ('#DataScience', 1),
 ('#AI', 1),
 ('#BigData', 1),
 ('#PySpark', 1),
 ('#BigData', 1),
 ('#AI', 1)]

In [16]:
# Count Hashtag Occurrences: Use countByValue to count how many times each hashtag appears.
hashtag_counts = tuples_hashtags_rdd.countByValue()
hashtag_counts

defaultdict(int,
            {('#BigData', 1): 4,
             ('#PySpark', 1): 3,
             ('#AI', 1): 5,
             ('#MachineLearning', 1): 1,
             ('#BigData.', 1): 1,
             ('#DataScience', 1): 2})

In [23]:
# Group Hashtags by Length: Use groupByKey to group hashtags by their length (number of characters).
hashtag_pairs_rdd = hashtags_rdd.map(lambda hashtag: (len(hashtag), hashtag))
hashtags_by_length = (hashtag_pairs_rdd.groupByKey().mapValues(list).collectAsMap())
for length, hashtags in sorted(hashtags_by_length.items()):
    print(f"Length {length}: {hashtags}")

Length 3: ['#AI', '#AI', '#AI', '#AI', '#AI']
Length 8: ['#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData']
Length 9: ['#BigData.']
Length 12: ['#DataScience', '#DataScience']
Length 16: ['#MachineLearning']
